In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wether/test_data.csv
/kaggle/input/wether/train.csv
/kaggle/input/wether/sample_submission (1) (2).csv


In [15]:
import pandas as pd
data_set = pd.read_csv("/kaggle/input/wether/train.csv")
data_set

,DATE,ACMH,ACSH,SNOW,PRCP,SNWD,TMAX,TMIN,TAVG,FMTM
0,1970-01-01,80.0,90.0,0.0,0.00,0.0,28,22,NaN,NaN
1,1970-01-02,30.0,20.0,0.0,0.00,0.0,31,22,NaN,NaN
2,1970-01-03,80.0,100.0,0.0,0.02,0.0,38,25,NaN,NaN
3,1970-01-04,10.0,20.0,0.0,0.00,0.0,31,23,NaN,NaN
4,1970-01-05,30.0,10.0,0.0,0.00,0.0,35,21,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
18988,2021-12-27,NaN,NaN,0.0,0.03,0.0,39,32,37.0,NaN
18989,2021-12-28,NaN,NaN,0.0,0.06,0.0,47,37,41.0,NaN
18990,2021-12-29,NaN,NaN,0.0,0.35,0.0,45,41,43.0,NaN
18991,2021-12-30,NaN,NaN,0.0,0.09,0.0,50,43,45.0,NaN


In [16]:
data_set = data_set.fillna(value=data_set["DATE"].mode())
data_set = data_set.fillna(value=data_set["ACSH"].mean())
data_set = data_set.fillna(value=data_set["SNOW"].mean())
data_set = data_set.fillna(value=data_set["PRCP"].mean())
data_set = data_set.fillna(value=data_set["SNWD"].mean())
data_set = data_set.fillna(value=data_set["TMAX"].mean())
data_set = data_set.fillna(value=data_set["TMIN"].mean())
data_set = data_set.fillna(value=data_set["TAVG"].mean())
data_set = data_set.fillna(value=data_set["FMTM"].mean())
data_set


,DATE,ACMH,ACSH,SNOW,PRCP,SNWD,TMAX,TMIN,TAVG,FMTM
0,1970-01-01,80.000000,90.000000,0.0,0.00,0.0,28,22,60.665557,60.665557
1,1970-01-02,30.000000,20.000000,0.0,0.00,0.0,31,22,60.665557,60.665557
2,1970-01-03,80.000000,100.000000,0.0,0.02,0.0,38,25,60.665557,60.665557
3,1970-01-04,10.000000,20.000000,0.0,0.00,0.0,31,23,60.665557,60.665557
4,1970-01-05,30.000000,10.000000,0.0,0.00,0.0,35,21,60.665557,60.665557
...,...,...,...,...,...,...,...,...,...,...
18988,2021-12-27,60.665557,60.665557,0.0,0.03,0.0,39,32,37.000000,60.665557
18989,2021-12-28,60.665557,60.665557,0.0,0.06,0.0,47,37,41.000000,60.665557
18990,2021-12-29,60.665557,60.665557,0.0,0.35,0.0,45,41,43.000000,60.665557
18991,2021-12-30,60.665557,60.665557,0.0,0.09,0.0,50,43,45.000000,60.665557


In [17]:
data_set['DATE'] = pd.to_datetime(data_set['DATE'], format='%Y-%m-%d')

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Assuming X and y are already defined

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('feature_selection', SelectFromModel(RandomForestRegressor(), threshold='median')),  # Select features
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))  # Random Forest Regressor
])

# Define parameter grid for GridSearchCV
param_grid = {
    # Define hyperparameters to search
    'regressor__n_estimators': [100, 200, 300],
    # Add more hyperparameters as needed
}

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X, y)

# Get best model
best_model = grid_search.best_estimator_

# Make predictions
y_pred = best_model.predict(X)

# Calculate mean squared error
mse = mean_squared_error(y, y_pred)

print("Best Parameters:", grid_search.best_params_)
print("Mean Squared Error:", mse)


Best Parameters: {'regressor__n_estimators': 300}
Mean Squared Error: 8.895063133526255


In [19]:
test_data = pd.read_csv("/kaggle/input/wether/test_data.csv")
test_data = test_data.fillna(value=test_data["ACSH"].mean())
test_data = test_data.fillna(value=test_data["SNOW"].mean())
test_data = test_data.fillna(value=test_data["PRCP"].mean())
test_data = test_data.fillna(value=test_data["SNWD"].mean())
test_data = test_data.fillna(value=test_data["TMIN"].mean())
test_data = test_data.fillna(value=test_data["TAVG"].mean())
test_data = test_data.fillna(value=test_data["FMTM"].mean())
test_data['DATE'] = pd.to_datetime(test_data['DATE'], format='%Y-%m-%d')

X_test = test_data.drop(columns=['DATE'])  
y_pred_test = model.predict(X_test)

# Create a DataFrame with DATE and predicted TMAX
forecast_results = pd.DataFrame({
    'ID': range(len(test_data)),
    'TMAX': y_pred_test
})

# Save the results to a CSV file
forecast_results.to_csv("forecast_results.csv", index=False)
forecast_results

,ID,TMAX
0,0,54.690000
1,1,58.350833
2,2,42.505737
3,3,30.837333
4,4,41.965109
...,...,...
289,289,67.588333
290,290,59.967845
291,291,56.615671
292,292,60.965032
